In [4]:
#地天板选股_问财接口

import pandas
import numpy
import datetime
from datetime import timedelta

def limit_down_to_up(start_date, end_date):
    dataframe=pd.DataFrame()
    date_range=get_trade_days(start_date, end_date)
    target_date=datetime.datetime.strptime(end_date,'%Y-%m-%d')
    end_date_next=(target_date+ timedelta(days=3)).strftime("%Y-%m-%d")
    date_range_next=get_trade_days(start_date, end_date_next)
    i=0
    for current_date in date_range: 
        i=i+1
        current_date_str=current_date.strftime("%Y-%m-%d")
        df=query_iwencai(current_date_str+" 地天板")
        if df.empty==False:
            df['交易日期']=current_date
            dataframe_quote=pd.DataFrame()
            for ticker in df['股票代码']:
                df_quote_1d=get_price(ticker,None,date_range_next[i],'1d',['quote_rate'],True,None,1)
                df_quote_1d['股票代码']=ticker
                dataframe_quote=dataframe_quote.append(df_quote_1d,ignore_index=True)
            df=pd.merge(df,dataframe_quote,on='股票代码')
            dataframe=dataframe.append(df,ignore_index=True)
    dataframe['次日涨跌幅']=dataframe['quote_rate']
    dataframe.drop('quote_rate', axis=1)
    order=['交易日期','股票代码','股票简称','涨停类型','次日涨跌幅']
    dataframe=dataframe.reindex(columns=order)
    return dataframe
df=limit_down_to_up("2023-01-01", "2023-10-20")
df
df.to_excel('2023年地天板.xlsx',sheet_name='Sheet1')

In [ ]:
#策略1选股_问财接口

import pandas
import numpy

start_date="2023-01-01"
end_date="2023-10-19"
def order_volume(start_date, end_date):
    dataframe=pd.DataFrame()
    date_range=get_trade_days(start_date, end_date)
    for current_date in date_range:
        current_date_str=current_date.strftime("%Y-%m-%d")
        str_list=[""," 09:45前成交额排序, ","涨跌幅,5日均线斜率>-10，","前3个交易日区间涨幅，", "后1个交易日最大涨幅,", "后1个交易日涨跌幅,", "前3个交易日区间涨幅在-10%和15%之间"]
        df=query_iwencai(current_date_str.join(str_list))
        dataframe=dataframe.append(df.head(2),ignore_index=True)
    return dataframe
dataframe
df.to_excel('2023年选股.xlsx',sheet_name='Sheet1')

import pandas
import numpy

start_date="2023-01-01"
end_date="2023-10-19"
def order_volume(start_date, end_date):
    dataframe=pd.DataFrame()
    date_range=get_trade_days(start_date, end_date)
    for current_date in date_range:
        current_date_str=current_date.strftime("%Y-%m-%d")
        str_list=[""," 09:45前成交额排序, ","涨跌幅,5日均线斜率>-10，","前3个交易日区间涨幅，", "后1个交易日最大涨幅,", "前3个交易日区间涨幅在-10%和15%之间"]
        df1=query_iwencai(current_date_str.join(str_list))
        trade_days=get_all_trade_days()
        index=(trade_days==current_date).argmax()
        next_date=trade_days[index+1]
        next_date_str=next_date.strftime("%Y-%m-%d")
        df2=query_iwencai(next_date_str+" 的涨幅")
        df=pd.merge(df1,df2,on='股票代码')
        df['日期']=current_date_str
        dataframe=dataframe.append(df.head(2),ignore_index=True)
    return dataframe
df=order_volume(start_date, end_date)
df
# df.to_excel('2023年选股.xlsx',sheet_name='Sheet1')




In [ ]:
#热股叠加策略1_问财接口
#可后续回测热股与胜率相关性

import pandas
import numpy
import datetime
from datetime import timedelta

startDate="2023-01-01"
endDate="2023-10-19"
timeEnd="09:45"
def orderVolume(startDate, endDate, timeEnd):
    dataframe=pd.DataFrame()
    dateRange=get_trade_days(startDate, endDate)
    for currentDate in dateRange:
        currentDateStr=currentDate.strftime("%Y-%m-%d")
        strList=[""," "+timeEnd+"前成交额排序, ","涨跌幅,5日均线斜率>-10，","前3个交易日区间涨幅，", "后1个交易日最大涨幅,", "前3个交易日区间涨幅在-10%和15%之间"]
        df1=query_iwencai(currentDateStr.join(strList))
        tradeDays=get_all_trade_days()
        index=(tradeDays==currentDate).argmax()
        nextDate=tradeDays[index+1]
        nextDateStr=nextDate.strftime("%Y-%m-%d")
        df2=query_iwencai(nextDateStr+" 的涨幅")
        df=pd.merge(df1,df2,on='股票代码')
        df['日期']=currentDateStr
        dataframe=dataframe.append(df.head(2),ignore_index=True)
    dataframe=dataframe.drop('股票简称_y',axis=1)
    colDict={
        "分时区间成交额":"截至"+time_end+"成交额",
        "分时区间成交额排名":"截至"+time_end+"成交额排名",
        "分时区间成交额排名名次":"截至"+time_end+"成交额排名名次",
        "分时区间成交额排名基数":"截至"+time_end+"成交额排名基数",
        "区间涨跌幅:前复权":"前3个交易日区间涨幅:前复权",
        "均线角度":"五日均线斜率",
        "最大涨幅":"后一个交易日最大涨幅",
        "涨跌幅:前复权_x":"当日涨跌幅:前复权",
        "股票代码":"股票代码",
        "股票简称_x":"股票简称",
        "涨跌幅:前复权_y":"后一个交易日涨跌幅:前复权",
        "日期":"日期"
        }
    dataframe.rename(columns=colDict, inplace=True)
    dataframe=dataframe.iloc[:,[11,8,9,0,1,2,3,4,5,6,7,10]]
    return dataframe
df=orderVolume(startDate, endDate, timeEnd)
df
# df.to_excel('2023年选股.xlsx',sheet_name='Sheet1')

n=len(df.iloc[:,1])
winNum=len(df[df.后一个交易日最大涨幅>0])
df.loc[n]=np.nan
df.iloc[n,8]="一日持有胜率"
df.iloc[n,9]=str(winNum/n*100)+"%"
df['前一日热股排名']=np.nan
for i in range(n):
    currentDate=df.loc[i,"日期"]
    lastDate1=datetime.datetime.strptime(currentDate,'%Y-%m-%d')
    lastDate=(lastDate1 - timedelta(days=1)).strftime("%Y-%m-%d")
    df1=query_iwencai(lastDate+"热股")
    if df.iloc[i,1]==df1.loc[0,"股票代码"] or (df.iloc[i,1]==df1.loc[:,"股票代码"]).argmax()!=0:
        j=(df.iloc[i,1]==df1.loc[:,"股票代码"]).argmax()
        rank=df1.loc[j,"个股热度排名名次"]
        df.loc[i,'前一日热股排名']=lastDate+" "+ str(rank) + "/50"
    else:
        df.loc[i,'前一日热股排名']=lastDate+" 未在热股前五十内"
df
# df.to_excel('2023年选股_2.xlsx',sheet_name='Sheet1')    
    
    
    
    

In [ ]:
#5连扳后断板策略_问财接口

import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import calendar

MonthRange=range(1,11,1)
tradeDays=get_all_trade_days()

def Function1(MonthRange):
    dataframe=pd.DataFrame()
    for i in MonthRange:
        if i==1:
            startDate="2022-12-01"
            endDate="2023-2-28"
        else:
            startDate="2023-"+str(i-1)+"-01"
            endDate="2023-"+str(i+1)+"-"+str(calendar.monthrange(2023,i+1)[1])
        df=query_iwencai(str(i)+"月 >4板，非st")
        tradeRange=get_trade_days(startDate,endDate)
        for num in range(len(df['股票代码'])):
            ticker=df.loc[num,'股票代码']
            value=get_price(ticker, str(tradeRange[0]), str(tradeRange[-1]),'1d',['open','close','low','high','high_limit','quote_rate'],True,None,0,False)
            for m in range(len(value)-4):
                mIndex=value.index[m]
                t=0
                if value.loc[mIndex,'close']==value.loc[mIndex,'high_limit']:
                    n=m
                    nIndex=value.index[n]
                    while value.loc[nIndex,'close']==value.loc[nIndex,'high_limit']:
                            highestBreak=value.loc[nIndex,'high']
                            t=t+1
                            n=n+1
                            nIndex=value.index[n]
                            if n==range(len(value))[-1]:break
                    if t>4:
                        if highestBreak<value.loc[nIndex,'high']:
                            highestBreak=value.loc[nIndex,'high']
                        df.loc[num,'断板时间']=value.index[n]
                        df.loc[num,'断板前后最高价']=highestBreak
                        j=n
                        if j==range(len(value))[-1]:
                            df.loc[num,'断板回调到最低价的时间']=value.index[j]
                        else:
                            while value.loc[value.index[j+1],'low']<=value.loc[value.index[j],'low'] or value.loc[value.index[j+1],'low']/value.loc[value.index[j],'low']<=1.0002:
                                j=j+1
                                if j==range(len(value))[-1]:
                                    break
                            df.loc[num,'断板回调到最低价的时间']=value.index[j]
                            lowest=value.loc[value.index[j],'low']
                            df.loc[num,'回调最低价']=lowest
                            df.loc[num,'时间间隔']=j-n
                            df.loc[num,'回调百分比']=(lowest-highestBreak)/highestBreak
                            if j!=range(len(value))[-1] and j!=range(len(value))[-2]:
                                k=j+1
                                while value.loc[value.index[k],'high']<value.loc[value.index[k+1],'high'] or value.loc[value.index[k],'high']/value.loc[value.index[k+1],'high']<=1.0002:
                                    k=k+1
                                    if k==range(len(value))[-1]:
                                        break
                                highest=value.loc[value.index[k],'high']
                                df.loc[num,'第一次反弹的最高价']=highest
                                df.loc[num,'反弹的比率']=(highest-lowest)/lowest
                                df.loc[num,'第一次反弹的时间间隔']=k-j
                                df.loc[num,'第一次反弹结束时间']=value.index[k]
        dataframe=dataframe.append(df, ignore_index=True)
    return dataframe

    
    